In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [89]:
class VeggieVision:
    def __init__(self, base_model, input_shape=(224, 224, 3)):
        # Set parameters
        self.input_shape = input_shape
        self.base_model = base_model
        self.model = None
        self.datagen = None

        # Build the model
        self.build_model()

    def build_model(self):
        # Freeze the base model layers initially
        self.base_model.trainable = False

       # 2. Build the Model
        self.model = models.Sequential()

        # Add the base model (MobileNetV2)
        self.model.add(self.base_model)

        self.model.add(layers.Flatten())

        # Add an additional Dense layer with 128 neurons and ReLU activation
        self.model.add(layers.Dense(256, activation='relu'))
        
        self.model.add(layers.Dropout(0.2))

        self.model.add(layers.Dense(256, activation = "relu"))

        # Optional: Add a Dropout layer to prevent overfitting
        self.model.add(layers.Dropout(0.2))

        #   Final Dense layer with a single neuron for regression (predicting weight)
        self.model.add(layers.Dense(1, activation='linear'))
        

    def augment_data(self, X_train):
        # 5. Data Augmentation
        # Create an ImageDataGenerator for real-time data augmentation
        self.datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range=180,         # Rotate images by up to 90 degrees
            width_shift_range=0.1,     # Shift image horizontally by up to 20% of the width
            height_shift_range=0.1,    # Shift image vertically by up to 20% of the height
            shear_range=0.1,           # Shear the image by up to 10%
            zoom_range=0.1,            # Zoom into the image by up to 20%
            horizontal_flip=True,      # Randomly flip images horizontally
            fill_mode='nearest'        # Fill in missing pixels with the nearest filled value
        )

        # Fit the ImageDataGenerator on the training data
        self.datagen.fit(X_train)

    def fit(self, X, y, batch_size=32, epochs=20, learning_rate=1e-4, validation_split=0.2, fine_tune=False):
        # 4. Split Data into Training and Validation Sets
        X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=validation_split, random_state=42)

        # Augment the data
        self.augment_data(X_train)

        # Compile the model with the provided learning rate
        self.model.compile(optimizer=Adam(learning_rate=learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # 6. Train the Model
        # Use the data generator to augment and train the model
        history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                 validation_data=(X_val, y_val), 
                                 epochs=epochs)

        # Evaluate the model on the validation set
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after initial training: {test_mae}')

        # 8. Fine-tuning the model (if requested)
        if fine_tune:
            self.fine_tune(X_train, y_train, X_val, y_val, batch_size, epochs, learning_rate)

        return history

    def fine_tune(self, X_train, y_train, X_val, y_val, batch_size=32, epochs=10, fine_tune_learning_rate=1e-5):
        # Unfreeze the base MobileNetV2 model's layers
        self.base_model.trainable = True

        # Compile with a lower learning rate for fine-tuning
        self.model.compile(optimizer=Adam(learning_rate=fine_tune_learning_rate), 
                           loss='mean_squared_error', 
                           metrics=['mae'])

        # Fine-tune the model with augmented data
        fine_tune_history = self.model.fit(self.datagen.flow(X_train, y_train, batch_size=batch_size),
                                           validation_data=(X_val, y_val), 
                                           epochs=epochs)

        # Evaluate after fine-tuning
        test_loss, test_mae = self.model.evaluate(X_val, y_val)
        print(f'Test MAE after fine-tuning: {test_mae}')

        return fine_tune_history
    
    def evaluate(self, X, y):
        test_loss, test_mae = self.model.evaluate(X, y)
        print(f'MAE: {test_mae}')
        return test_loss, test_mae


In [88]:
X = np.load("../data/processed_data/X.npy")
y = np.load("../data/processed_data/y.npy")

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Load the VGG-16 model as the base model
base_model = VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

In [59]:
from tensorflow.keras.applications.resnet50 import preprocess_input
X_res = preprocess_input(X)

In [71]:
from tensorflow.keras.applications import ResNet50

# Load the ResNet50 model with pre-trained ImageNet weights, excluding the top layers.
base_model = ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

# Assuming VeggieVision is your custom model class that accepts a base model.
veggie_res = VeggieVision(base_model)

In [73]:
veggie_res.fit(X_res,y, learning_rate = 1e-4, fine_tune=True)

Epoch 1/20
10/10 [==============================] - 44s 4s/step - loss: 18292.0352 - mae: 74.6614 - val_loss: 5803.4937 - val_mae: 68.7640
Epoch 2/20
 8/10 [=======================>......] - ETA: 5s - loss: 19958.4668 - mae: 75.3330

KeyboardInterrupt: 

In [90]:
base_model = MobileNetV2(input_shape=(224,224, 3), include_top=False, weights='imagenet')
veggie_mnv2 = VeggieVision(base_model)

In [ ]:
veggie_mnv2.fit(X,y, learning_rate = 1e-5, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 17s 783ms/step - loss: 10290.0996 - mae: 32.7677 - val_loss: 1096.1611 - val_mae: 24.0153
Epoch 2/20
19/19 [==============================] - 14s 747ms/step - loss: 10376.5439 - mae: 31.6737 - val_loss: 1101.9620 - val_mae: 24.2148
Epoch 3/20
19/19 [==============================] - ETA: 0s - loss: 9525.5771 - mae: 32.2066

In [57]:
from tensorflow.keras.applications import EfficientNetB7
base_model = EfficientNetB7(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
veggie_e_ = VeggieVision(base_model)

In [44]:
veggie_smol_boi.fit(X, y, learning_rate= 1e-5, fine_tune = True)

Epoch 1/20
 2/10 [=====>........................] - ETA: 9s - loss: 22645.1133 - mae: 125.5711 

KeyboardInterrupt: 

In [33]:
veggie_big_boi_no_dropout.fit(X,y, learning_rate = 1e-6, fine_tune= True)

Epoch 1/20
10/10 [==============================] - 16s 1s/step - loss: 8191.3213 - mae: 27.3749 - val_loss: 855.2877 - val_mae: 20.5559
Epoch 2/20
10/10 [==============================] - 13s 1s/step - loss: 8600.0234 - mae: 26.9130 - val_loss: 855.4891 - val_mae: 20.5583
Epoch 3/20
10/10 [==============================] - 13s 1s/step - loss: 8752.1250 - mae: 26.0550 - val_loss: 856.2388 - val_mae: 20.5658
Epoch 4/20
10/10 [==============================] - 13s 1s/step - loss: 7974.8662 - mae: 26.3714 - val_loss: 857.7885 - val_mae: 20.5851
Epoch 5/20
10/10 [==============================] - 13s 1s/step - loss: 8083.3647 - mae: 26.4357 - val_loss: 857.6528 - val_mae: 20.5832
Epoch 6/20
10/10 [==============================] - 13s 1s/step - loss: 6804.3564 - mae: 24.7265 - val_loss: 857.4573 - val_mae: 20.5808
Epoch 7/20
10/10 [==============================] - 13s 1s/step - loss: 9161.2910 - mae: 26.5058 - val_loss: 856.8440 - val_mae: 20.5731
Epoch 8/20
10/10 [=======================

KeyboardInterrupt: 

In [34]:
veggie_big_boi_no_dropout.evaluate(X, y)

23/23 [==============================] - 11s 485ms/step - loss: 6443.5864 - mae: 24.2655
MAE: 24.26554298400879


(6443.58642578125, 24.26554298400879)

In [27]:
veggie_big_big_boi.fit(X,y, learning_rate = 1e-5, fine_tune=True)

Epoch 1/20
10/10 [==============================] - 17s 1s/step - loss: 9083.3281 - mae: 33.4053 - val_loss: 1144.6912 - val_mae: 23.9746
Epoch 2/20
10/10 [==============================] - 14s 1s/step - loss: 11365.5576 - mae: 32.1404 - val_loss: 1163.6149 - val_mae: 24.2507
Epoch 3/20
10/10 [==============================] - 14s 1s/step - loss: 10380.8340 - mae: 31.7568 - val_loss: 1152.2046 - val_mae: 24.0482
Epoch 4/20
10/10 [==============================] - 14s 1s/step - loss: 11973.4551 - mae: 31.6107 - val_loss: 1116.8442 - val_mae: 23.5158
Epoch 5/20


KeyboardInterrupt: 

In [18]:
veggie_big_boi.fit(X,y, learning_rate = 1e-6, fine_tune = True)

Epoch 1/20
19/19 [==============================] - 18s 773ms/step - loss: 7576.7842 - mae: 26.0752 - val_loss: 1261.8690 - val_mae: 23.1831
Epoch 2/20
19/19 [==============================] - 14s 739ms/step - loss: 8056.1162 - mae: 27.1340 - val_loss: 1261.1941 - val_mae: 23.1792
Epoch 3/20
19/19 [==============================] - 14s 743ms/step - loss: 6077.2007 - mae: 25.5753 - val_loss: 1259.8824 - val_mae: 23.1506
Epoch 4/20
19/19 [==============================] - 14s 730ms/step - loss: 9303.9893 - mae: 28.2925 - val_loss: 1260.5011 - val_mae: 23.1528
Epoch 5/20
19/19 [==============================] - 14s 735ms/step - loss: 10123.6045 - mae: 27.0236 - val_loss: 1257.7966 - val_mae: 23.1083
Epoch 6/20
 8/19 [===========>..................] - ETA: 6s - loss: 994.4074 - mae: 22.5590 

KeyboardInterrupt: 

In [11]:
veggie_no_pool.fit(X,y, learning_rate= 1e-5, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 18s 805ms/step - loss: 8651.3008 - mae: 28.6878 - val_loss: 1231.3660 - val_mae: 23.7288
Epoch 2/20
19/19 [==============================] - 14s 741ms/step - loss: 10208.2119 - mae: 27.9961 - val_loss: 1209.7805 - val_mae: 23.4710
Epoch 3/20
19/19 [==============================] - 15s 794ms/step - loss: 8565.7734 - mae: 27.7041 - val_loss: 1231.7828 - val_mae: 23.8033
Epoch 4/20
19/19 [==============================] - 15s 766ms/step - loss: 8362.1279 - mae: 28.6560 - val_loss: 1220.3612 - val_mae: 23.6964
Epoch 5/20
10/19 [==============>...............] - ETA: 5s - loss: 1103.9445 - mae: 23.9780

KeyboardInterrupt: 

In [ ]:
veggie.fit(X, y)

In [10]:
veggie_64 = VeggieVision()

In [14]:
veggie_64.fit(X, y, fine_tune=True, learning_rate=1e-6)

Epoch 1/20
19/19 [==============================] - 16s 734ms/step - loss: 12264.9004 - mae: 30.9180 - val_loss: 1256.3654 - val_mae: 25.2430
Epoch 2/20
19/19 [==============================] - 13s 706ms/step - loss: 12325.9336 - mae: 31.4613 - val_loss: 1256.3182 - val_mae: 25.2425
Epoch 3/20
19/19 [==============================] - 14s 728ms/step - loss: 11546.5693 - mae: 31.4237 - val_loss: 1256.1592 - val_mae: 25.2418
Epoch 4/20
19/19 [==============================] - 14s 721ms/step - loss: 12680.3691 - mae: 30.9341 - val_loss: 1256.1520 - val_mae: 25.2411
Epoch 5/20
19/19 [==============================] - 14s 714ms/step - loss: 12418.0391 - mae: 30.2319 - val_loss: 1256.0312 - val_mae: 25.2405
Epoch 6/20
19/19 [==============================] - 14s 732ms/step - loss: 12270.7178 - mae: 31.1645 - val_loss: 1256.0015 - val_mae: 25.2399
Epoch 7/20
19/19 [==============================] - 14s 732ms/step - loss: 11956.3135 - mae: 30.9879 - val_loss: 1255.9066 - val_mae: 25.2394
Epoch 

KeyboardInterrupt: 

In [21]:
veggie_2_layer = VeggieVision()

In [25]:
veggie_2_layer.fit(X, y, fine_tune=True, learning_rate= 1e-6)

Epoch 1/20
19/19 [==============================] - 17s 784ms/step - loss: 11959.4062 - mae: 31.8935 - val_loss: 1107.3799 - val_mae: 22.4722
Epoch 2/20
19/19 [==============================] - 13s 703ms/step - loss: 12233.4707 - mae: 30.9931 - val_loss: 1107.3876 - val_mae: 22.4720
Epoch 3/20
19/19 [==============================] - 14s 724ms/step - loss: 11918.9775 - mae: 31.2685 - val_loss: 1107.2454 - val_mae: 22.4707
Epoch 4/20
 3/19 [===>..........................] - ETA: 9s - loss: 1270.4844 - mae: 27.3488 

KeyboardInterrupt: 

In [28]:
veggie_2_layer.evaluate(X, y, 0.2)

23/23 [==============================] - 12s 515ms/step - loss: 9486.8643 - mae: 27.3929
Test MAE: 27.392892837524414


(9486.8642578125, 27.392892837524414)

In [30]:
veggie_2_big = VeggieVision()

In [35]:
veggie_2_big.fit(X, y, learning_rate=1e-6, fine_tune=True)

Epoch 1/20
19/19 [==============================] - 17s 736ms/step - loss: 11619.3389 - mae: 29.9503 - val_loss: 1095.9834 - val_mae: 23.1957
Epoch 2/20
19/19 [==============================] - 14s 710ms/step - loss: 11783.5479 - mae: 29.6712 - val_loss: 1096.0830 - val_mae: 23.1945
Epoch 3/20
19/19 [==============================] - 14s 709ms/step - loss: 11432.7910 - mae: 30.2055 - val_loss: 1095.8263 - val_mae: 23.1918
Epoch 4/20
19/19 [==============================] - 13s 703ms/step - loss: 11687.8711 - mae: 29.4996 - val_loss: 1095.8114 - val_mae: 23.1911
Epoch 5/20
19/19 [==============================] - 14s 725ms/step - loss: 11812.0664 - mae: 29.9659 - val_loss: 1095.5537 - val_mae: 23.1903
Epoch 6/20
19/19 [==============================] - 14s 702ms/step - loss: 11736.2432 - mae: 29.2203 - val_loss: 1095.7441 - val_mae: 23.1918
Epoch 7/20
19/19 [==============================] - 14s 724ms/step - loss: 11217.1807 - mae: 30.2717 - val_loss: 1095.5061 - val_mae: 23.1897
Epoch 

KeyboardInterrupt: 

In [37]:
veggie_2_big.evaluate(X, y)

23/23 [==============================] - 12s 495ms/step - loss: 9496.3623 - mae: 27.8528
MAE: 27.852787017822266


(9496.3623046875, 27.852787017822266)